In [1]:
# Set up enviroment
import json
import re
import pandas as pd
import random
from dotenv import load_dotenv
from llama_index.core import (
    Document,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
    get_response_synthesizer,
)
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.evaluation import RetrieverEvaluator
from llama_index.core.prompts import PromptTemplate
from llama_index.core.schema import QueryBundle
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Key
load_dotenv()
# Set embedding model
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [2]:
# Carga de las preguntas
qa_path = "../data/exams/clean_questions/questions_all_tests.json"
with open(qa_path, "r", encoding="utf-8") as f:
    questions = json.load(f)

In [3]:
# Ruta al índice optimizado
storage_path = "../data/index_storage_200_90/"
storage_context = StorageContext.from_defaults(persist_dir=storage_path)
index = load_index_from_storage(storage_context)

# Configurar el retriever
retriever = index.as_retriever(similarity_top_k=5)

In [4]:
# Función auxiliar para formatear las opciones:
def format_options(options_dict):
    clean_options = {}
    for key in options_dict:
        # Elimina letras errantes como '\nB' o '\nC' dentro del texto
        value = options_dict[key]
        value = re.sub(r'\n[A-E]\n?', '', value).strip()
        clean_options[key] = value
    return "\n".join([f"{k}. {v}" for k, v in clean_options.items()])

# Validar la repsuesta:
def display_question_result(result):
    print("ID Pregunta:", result["question_id"])
    print("\nPregunta:\n", result["question"])
    print("\nOpciones:\n", result["options"])
    print("\nAcertó?:", result["is_correct"])
    print("\nRespuesta generada por el modelo:\n", result["response_obj"])
    print("\nRespuesta correcta:", result["correct_answer"])
    if "response_obj" in result:
        print(f"\n🔎 Chunks usados para responder la pregunta {result['question_id']}:\n")
        for i, node in enumerate(result["response_obj"].source_nodes):
            print(f"--- Chunk #{i+1} | Score: {node.score:.4f} ---")
            print(node.node.get_content().strip())
            print()

system_prompt = """
Eres un experto en Apache Spark y PySpark, y estás respondiendo preguntas tipo test usando exclusivamente la información del contexto proporcionado.

No puedes utilizar ningún conocimiento previo ni información externa al contexto.

Tu tarea es:
1. Leer el contexto cuidadosamente.
2. Leer la pregunta y las opciones tipo test.
3. Seleccionar la opción correcta si está claramente justificada por el contenido del contexto, y justificar tu elección citando fragmentos textuales exactos.
4. Si no hay una justificación directa, puedes razonar una respuesta **únicamente si se basa en información presente en el contexto**, indicando exactamente **qué fragmentos usas** y **cómo los conectas**. No está permitido inventar datos.
5. En caso de no encontrar ninguna información útil en el contexto, responde exactamente con: **"No existe información"**
"""

llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0,
    system_prompt=system_prompt
)

Settings.llm = llm

def extract_predicted_letter(response_text):
    """
    Extrae la letra (A, B, C, D o E) de una respuesta generada en el formato:
    'Respuesta correcta: <LETRA>\nJustificación: <...>'
    """
    match = re.search(r"Respuesta\s+correcta\s*:\s*([A-E])", response_text, flags=re.IGNORECASE)
    if match:
        return match.group(1).upper()
    return None 

def ask_question_with_options(question_obj, 
                              #query_str_template, 
                              retriever = retriever, response_mode= "compact"):
    question_text = question_obj["question"]
    options_text = format_options(question_obj["options"])

    #qa_prompt = PromptTemplate(query_str_template)
    #Settings.text_qa_template = qa_prompt

    query_str = f"""
    {question_text}

    Opciones: 
    {options_text}
        
    IMPORTANTE:
        - Usa solo información del contexto.
        - No respondas con conocimientos externos.
        - Aunque sepas la respuesta correcta, si no existe esa información en el contexto devuelve "No existe información"
    
    Responde de la siguiente manera:
    "
    Respuesta correcta: <LETRA>
    Justificación: <JUSTIFICACIÓN>
    "
    """
    
    # Usar QueryBundle para pasar la información
    query_bundle = QueryBundle(
        query_str = query_str,
        custom_embedding_strs = [question_text, options_text]
    )

    response_synthesizer = get_response_synthesizer(response_mode=response_mode)
    
    query_engine = RetrieverQueryEngine(
        retriever=retriever,
        response_synthesizer=response_synthesizer
    )
    
    response = query_engine.query(query_bundle)
    predicted_answer = response.response.strip()
    
    return {
        "question_id": question_obj["question_id"],
        "question" : question_obj["question"],
        "options": format_options(question_obj["options"]),
        "predicted_answer": predicted_answer,
        "correct_answer": question_obj["correct_answer"],
        "is_correct": extract_predicted_letter(predicted_answer) == question_obj["correct_answer"],
        "response_obj": response
    }

In [5]:
questions_to_evaluate = random.sample(questions, 5)

In [6]:
for q in questions_to_evaluate:
    response = ask_question_with_options(q, response_mode = "tree_summarize")
    display_question_result(response)

ID Pregunta: Test_1_41

Pregunta:
 The following code block contains an error. The code block should return the df DataFrame with 
employeeID renamed to employeeIdColumn. Find the error.
df.withColumn("employeeIdColumn", "employeeID")

Opciones:
 A. Instead of withColumn, the withColumnRenamed method should be used
B. Instead of withColumn, the withColumnRenamed method should be used and 
argument "employeeIdColumn" should be swapped with argument "employeeID"
C. Arguments "employeeIdColumn" and "employeeID" should be swapped
D. The withColumn operator should be replaced with the withColumnRenamed operator

Acertó?: False

Respuesta generada por el modelo:
 Respuesta correcta: A  
Justificación: El contexto menciona que "para cambiar el nombre de una columna, se utilizaría la función withColumnRenamed()". Esto indica que el uso de withColumn en el código proporcionado es incorrecto para renombrar una columna, y por lo tanto, se debe utilizar withColumnRenamed en su lugar.

Respuesta co